# Puzzle generation

In [36]:
import numpy as np
import json
from typing import List, Tuple

NUMBER_OF_EASY = 100
NUMBER_OF_MEDIUM = 100
NUMBER_OF_HARD = 100

def extract_constraints(grid: np.ndarray) -> Tuple[List[List[int]], List[List[int]]]:
    def get_line_constraints(line):
        constraints = []
        count = 0
        for val in line:
            if val == 1:
                count += 1
            elif count > 0:
                constraints.append(count)
                count = 0
        if count > 0:
            constraints.append(count)
        return constraints or [0]

    row_constraints = [get_line_constraints(row) for row in grid]
    col_constraints = [get_line_constraints(col) for col in grid.T]
    return row_constraints, col_constraints

def generate_puzzles(size: int, count: int) -> List[dict]:
    puzzles = []
    while len(puzzles) < count:
        solution = np.random.randint(0, 2, (size, size))
        row_cons, col_cons = extract_constraints(solution)
        puzzles.append({
            "row": row_cons,
            "col": col_cons,
            "solution": solution.tolist()
        })
    return puzzles

def save_to_file(puzzles: List[dict], filename: str):
    with open(filename, 'w') as f:
        for puzzle in puzzles:
            f.write(json.dumps(puzzle) + '\n')


save_to_file(generate_puzzles(5, NUMBER_OF_EASY), "small.txt")
save_to_file(generate_puzzles(10, NUMBER_OF_MEDIUM), "medium.txt")
save_to_file(generate_puzzles(15, NUMBER_OF_HARD), "large.txt")


# Code for getting solution statistics

In [37]:
import numpy as np
from numpy.typing import NDArray
import json

def check(grid: np.ndarray, row_constraints, col_constraints) -> bool:
    def get_blocks(line) -> list[int]:
        return [sum(g) for g in np.split(line, np.where(line == 0)[0]) if sum(g) > 0]

    for i, row in enumerate(grid):
        if get_blocks(row) != row_constraints[i]:
            return False
    for j, col in enumerate(grid.T):
        if get_blocks(col) != col_constraints[j]:
            return False
    return True

def get_blocks(line: np.ndarray) -> list[int]:
    blocks = []
    count = 0
    for cell in line:
        if cell == 1:
            count += 1
        elif count:
            blocks.append(count)
            count = 0
    if count:
        blocks.append(count)
    return blocks

def puzzle_accuracy(grid: NDArray[np.int_], row_constraints: list[list[int]], col_constraints: list[list[int]]) -> float:
    """Proportion of rows and columns that match their constraints."""
    grid_size = grid.shape[0]
    correct_rows = sum(get_blocks(grid[i]) == row_constraints[i] for i in range(grid_size))
    correct_cols = sum(get_blocks(grid[:, j]) == col_constraints[j] for j in range(grid_size))
    return (correct_rows + correct_cols) / (2 * grid_size)

def constraints_accuracy(grid: NDArray[np.int_], row_constraints: list[list[int]], col_constraints: list[list[int]]) -> float:
    """Proportion of individual constraint elements (numbers in row/col specs) that match."""
    matched = 0
    total = 0

    for i in range(grid.shape[0]):
        actual = get_blocks(grid[i])
        expected = row_constraints[i]
        total += len(expected)
        matched += sum(1 for a, b in zip(actual, expected) if a == b)

    for j in range(grid.shape[1]):
        actual = get_blocks(grid[:, j])
        expected = col_constraints[j]
        total += len(expected)
        matched += sum(1 for a, b in zip(actual, expected) if a == b)

    if total == 0:
        return 1.0  # no constraints to match
    return matched / total


def save_result(results, filename: str):
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)


# Genetic Algorithm

In [38]:
import random
import numpy as np
from numpy.typing import NDArray
import time
import json

# Set global vars for the solver
mutation_rate = 0.01
generations = 100
population_size = 100

# Load puzzles
with open("small.txt") as f:
    small = [json.loads(line) for line in f]
with open("medium.txt") as f:
    medium = [json.loads(line) for line in f]
with open("large.txt") as f:
    large = [json.loads(line) for line in f]

results = {"small": [], "medium": [], "large": []}

def create_individual() -> list[int]:
    """Generate a random individual (binary grid representation)."""
    return [random.randint(0, 1) for _ in range(grid_size**2)]

def evaluate(individual: list[int]) -> int:
    """Evaluate the fitness of an individual based on row and column constraints."""
    grid = np.array(individual).reshape((grid_size, grid_size))
    score = 0  # Higher score means a better match to constraints

    # Evaluate row constraints
    for i, row in enumerate(grid):
        blocks = [sum(g) for g in np.split(row, np.where(row == 0)[0]) if sum(g) > 0]
        if blocks == row_constraints[i]:  
            score += 1  

    # Evaluate column constraints
    for j, col in enumerate(grid.T):  # Transpose to iterate over columns
        blocks = [sum(g) for g in np.split(col, np.where(col == 0)[0]) if sum(g) > 0]
        if blocks == col_constraints[j]:  
            score += 1  

    return score  # Higher score means better constraint satisfaction

def select(population: list[list[int]]) -> list[list[int]]:
    """Select the top-performing individuals for reproduction."""
    population.sort(key=evaluate, reverse=True)  # Sort by fitness (descending order)
    return population[:population_size // 2]  # Retain the top 50% of individuals

def crossover(parent1: list[int], parent2: list[int]) -> tuple[list[int], list[int]]:
    """Perform single-point crossover between two parents."""
    point = random.randint(0, grid_size**2 - 1)  # Choose a crossover point
    child1 = parent1[:point] + parent2[point:]  
    child2 = parent2[:point] + parent1[point:]  
    return child1, child2

def mutate(individual: list[int]) -> None:
    """Apply mutation to an individual with a certain probability."""
    for i in range(len(individual)):
        if random.random() < mutation_rate:  
            individual[i] = 1 - individual[i]  # Flip the bit (0 → 1, 1 → 0)

def genetic_algorithm() -> NDArray[np.long]:
    """Solve the Picross puzzle using a genetic algorithm."""
    population = [create_individual() for _ in range(population_size)]

    for generation in range(generations):
        population = select(population)  # Select the fittest individuals

        # If an individual satisfies all constraints, terminate early
        if evaluate(population[0]) == grid_size * 2:
            break

        next_generation = population.copy()

        # Generate offspring until the population size is restored
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)  # Select two parents
            child1, child2 = crossover(parent1, parent2)  # Perform crossover
            mutate(child1)  # Apply mutation
            mutate(child2)  
            next_generation.extend([child1, child2])  # Add offspring to the population
        
        population = next_generation[:population_size]  # Ensure population size remains constant
        
        best_individual = max(population, key=evaluate)  # Track the best solution found

    # Display the best solution found
    best_solution = np.array(best_individual).reshape((grid_size, grid_size))
    
    return best_solution

# Run the genetic algorithm to solve the Picross puzzle
for size_name, puzzles in [("small", small), ("medium", medium), ("large", large)]:
    for puzzle in puzzles:
        row_constraints = puzzle["row"]
        col_constraints = puzzle["col"]
        grid_size = len(row_constraints)

        start = time.time()
        grid = genetic_algorithm()
        end = time.time()
        print(end - start)
        
        correct = check(grid, row_constraints, col_constraints)
        p_acc = puzzle_accuracy(grid, row_constraints, col_constraints)
        c_acc = constraints_accuracy(grid, row_constraints, col_constraints)

        results[size_name].append({
            "solved": correct,
            "time": end - start,
            "puzzle_accuracy": p_acc,
            "constraints_accuracy": c_acc
        })

# Save results
save_result(results, "GA-results.json")


0.6330671310424805
1.8854293823242188
1.9158680438995361
1.903019905090332
1.785736083984375
0.21223735809326172
1.712364673614502
1.821483850479126
1.829293966293335
1.8294594287872314
3.053882360458374
5.042116165161133
4.998175859451294
5.088141918182373
4.745753765106201
4.865400791168213
0.683189868927002
4.995218276977539
0.9437885284423828
4.939810276031494
0.6428515911102295
4.461163759231567
1.0003530979156494
1.1071183681488037
4.895379066467285
4.93649435043335
5.012935161590576
4.9130425453186035
4.5125954151153564
0.9229059219360352
0.6167294979095459
4.683903217315674
4.776459217071533
4.933189392089844
4.86105751991272
4.715180397033691
4.847665786743164
4.364474296569824
4.62905216217041
4.225018501281738
1.0355236530303955
0.7983126640319824
3.050358533859253
4.780516862869263
4.80664324760437
4.703656196594238
4.853530645370483
4.921757936477661
0.748272180557251
4.894109487533569
4.663888931274414
4.779879331588745
4.731560468673706
4.435694456100464
4.63237118721008

# Simulated Annealing

In [40]:
import random
import numpy as np
from numpy.typing import NDArray
import time
import json

# Set global vars for the solver
initial_temperature = 10.0
cooling_rate = 0.999
iterations = 10000

# Load puzzles
with open("small.txt") as f:
    small = [json.loads(line) for line in f]
with open("medium.txt") as f:
    medium = [json.loads(line) for line in f]
with open("large.txt") as f:
    large = [json.loads(line) for line in f]

results = {"small": [], "medium": [], "large": []}

def evaluate(grid: NDArray[np.long]) -> int:
    """
    Calculate the fitness score of a grid based on row and column constraints.
    A higher score indicates a better match to the constraints.
    """
    score = 0  # Initialize score
    
    # Evaluate row constraints
    for i, row in enumerate(grid):
        # Find contiguous blocks of 1s in the row
        blocks = [sum(g) for g in np.split(row, np.where(row == 0)[0]) if sum(g) > 0]
        if blocks == row_constraints[i]:  
            score += 1  

    # Evaluate column constraints
    for j, col in enumerate(grid.T):  # Transpose to iterate over columns
        blocks = [sum(g) for g in np.split(col, np.where(col == 0)[0]) if sum(g) > 0]
        if blocks == col_constraints[j]:  
            score += 1  

    return score  # Higher score means better constraint satisfaction

def mutate(grid: NDArray[np.long]) -> NDArray[np.long]:
    """
    Create a new grid by flipping a random cell (mutating the grid).
    """
    new_grid = grid.copy()  # Copy the current grid
    i, j = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)  # Select a random cell
    new_grid[i, j] = 1 - new_grid[i, j]  # Flip the bit (0 → 1, 1 → 0)
    return new_grid

def simulated_annealing() -> NDArray[np.long]:
    """
    Solve the Picross puzzle using the simulated annealing algorithm.
    """
    # Initialize a random grid with binary values (0 or 1)
    current_grid = np.random.randint(0, 2, (grid_size, grid_size))
    current_score = evaluate(current_grid)  # Compute its initial fitness score
    temperature = initial_temperature  # Set the initial temperature
    
    # Keep track of the best solution found
    best_grid, best_score = current_grid, current_score

    for _ in range(iterations):
        # Generate a new candidate solution by mutating the current grid
        new_grid = mutate(current_grid)
        new_score = evaluate(new_grid)

        # If the new grid is better, accept it unconditionally
        if new_score > current_score:
            current_grid, current_score = new_grid, new_score
        else:
            # Otherwise, accept with a probability determined by the temperature
            delta = new_score - current_score
            if random.random() < np.exp(delta / temperature):  
                current_grid, current_score = new_grid, new_score

        # Update the best solution if the new one is the best so far
        if new_score > best_score:
            best_grid, best_score = new_grid, new_score

        # Reduce the temperature over time
        temperature *= cooling_rate

        # Stop early if a perfect solution is found
        if best_score == grid_size * 2:
            break

    return best_grid

# Run the simulated annealing algorithm to solve the Picross puzzle
for size_name, puzzles in [("small", small), ("medium", medium), ("large", large)]:
    for puzzle in puzzles:
        row_constraints = puzzle["row"]
        col_constraints = puzzle["col"]
        grid_size = len(row_constraints)

        start = time.time()
        grid = simulated_annealing()
        end = time.time()
        print(end - start)
        correct = check(grid, row_constraints, col_constraints)
        p_acc = puzzle_accuracy(grid, row_constraints, col_constraints)
        c_acc = constraints_accuracy(grid, row_constraints, col_constraints)

        results[size_name].append({
            "solved": correct,
            "time": end - start,
            "puzzle_accuracy": p_acc,
            "constraints_accuracy": c_acc
        })

# Save results
save_result(results, "SA-results.json")


0.32692718505859375
0.31182169914245605
0.9856374263763428
0.283750057220459
0.272083044052124
0.3284609317779541
0.9161536693572998
0.9077167510986328
0.9064040184020996
0.2858309745788574
0.9704697132110596
0.9694933891296387
0.9481141567230225
0.9632155895233154
0.312969446182251
0.9377408027648926
0.2985360622406006
0.9362471103668213
0.3170609474182129
0.9503841400146484
0.1990666389465332
0.31682682037353516
0.849311351776123
0.928382158279419
0.9297599792480469
0.9338843822479248
0.9429404735565186
0.3868982791900635
0.3299980163574219
0.31830549240112305
0.9134984016418457
0.9227838516235352
0.9712138175964355
0.9491674900054932
0.9418718814849854
0.9130692481994629
0.9202053546905518
0.2958641052246094
0.9042556285858154
0.30275726318359375
0.2649269104003906
0.30213499069213867
0.82979416847229
0.28852272033691406
0.2939131259918213
0.8736367225646973
0.31279420852661133
0.9458913803100586
0.2779991626739502
0.980964183807373
0.3144083023071289
0.9200007915496826
0.8761286735

# Results Analysis

In [41]:
import json

def analyze(file: str):
    with open(file) as f:
        data = json.load(f)

    for size, records in data.items():
        total = len(records)
        solved = sum(1 for r in records if r["solved"])
        avg_time = sum(r["time"] for r in records) / max(total, 1)
        avg_puzzle_acc = sum(r.get("puzzle_accuracy", 0) for r in records) / max(total, 1)
        avg_constraint_acc = sum(r.get("constraints_accuracy", 0) for r in records) / max(total, 1)

        print(f"{file} - {size}:")
        print(f"  Accuracy: {solved / max(total, 1) * 100:.1f}%")
        print(f"  Avg puzzle accuracy: {avg_puzzle_acc * 100:.2f}%")
        print(f"  Avg constraint accuracy: {avg_constraint_acc * 100:.2f}%")
        print(f"  Avg time: {avg_time:.4f}s\n")

analyze("GA-results.json")
analyze("SA-results.json")

GA-results.json - small:
  Accuracy: 30.0%
  Avg puzzle accuracy: 86.50%
  Avg constraint accuracy: 88.72%
  Avg time: 2.7817s

GA-results.json - medium:
  Accuracy: 0.0%
  Avg puzzle accuracy: 50.05%
  Avg constraint accuracy: 63.08%
  Avg time: 6.3423s

GA-results.json - large:
  Accuracy: 0.0%
  Avg puzzle accuracy: 26.20%
  Avg constraint accuracy: 47.21%
  Avg time: 19.3568s

SA-results.json - small:
  Accuracy: 49.0%
  Avg puzzle accuracy: 92.40%
  Avg constraint accuracy: 94.12%
  Avg time: 0.6222s

SA-results.json - medium:
  Accuracy: 0.0%
  Avg puzzle accuracy: 72.25%
  Avg constraint accuracy: 78.82%
  Avg time: 2.6422s

SA-results.json - large:
  Accuracy: 0.0%
  Avg puzzle accuracy: 41.17%
  Avg constraint accuracy: 56.89%
  Avg time: 7.2647s

